# 🎯 Pipeline 1: Object Detection and Segmentation
## Problem-Solving Pipeline for Classical Computer Vision

### 🎯 Problem Statement:
**"Given an image with objects, detect and segment them using classical computer vision techniques"**

### 📋 Solution Approach:
1. **Preprocessing** → Noise reduction and enhancement
2. **Thresholding** → Separate objects from background
3. **Morphological Operations** → Clean up binary masks
4. **Contour Detection** → Find object boundaries
5. **Analysis** → Extract object properties

### 🔧 Techniques Used:
- Image enhancement (histogram equalization, contrast stretching)
- Smoothing filters (Gaussian, median)
- Adaptive thresholding
- Morphological operations (opening, closing)
- Contour detection and analysis

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def show_results(images, titles, rows=2, cols=3, figsize=(15, 10)):
    """Display multiple images in a grid"""
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten() if rows * cols > 1 else [axes]
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if i >= len(axes):
            break
        ax = axes[i]
        
        if len(img.shape) == 3:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            ax.imshow(img, cmap='gray')
        ax.set_title(title, fontweight='bold')
        ax.axis('off')
    
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

def create_test_objects():
    """Create a test image with objects for segmentation"""
    img = np.zeros((400, 400, 3), dtype=np.uint8)
    
    # Add various objects
    cv2.rectangle(img, (50, 50), (150, 150), (255, 100, 100), -1)  # Red rectangle
    cv2.circle(img, (300, 100), 50, (100, 255, 100), -1)          # Green circle
    cv2.ellipse(img, (200, 250), (80, 40), 45, 0, 360, (100, 100, 255), -1)  # Blue ellipse
    cv2.rectangle(img, (50, 300), (120, 370), (255, 255, 100), -1)  # Yellow rectangle
    
    # Add some noise
    noise = np.random.randint(0, 30, img.shape, dtype=np.uint8)
    img = cv2.add(img, noise)
    
    return img

print("🎯 Object Detection and Segmentation Pipeline Ready!")

## 📖 Step 1: Load and Analyze Input Image

In [ ]:
# Load or create test image
try:
    img = cv2.imread('D:/FPT_Material/Sem 4/CPV301/Source for PE/Image/image.jpg')
    if img is None:
        raise FileNotFoundError("Image not found")
    print("✅ Loaded image from file")
except:
    print("⚠️ Creating test image with objects...")
    img = create_test_objects()

# Convert to different formats
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

print(f"📊 Image Analysis:")
print(f"Shape: {img.shape}")
print(f"Data type: {img.dtype}")
print(f"Brightness range: [{gray_img.min()}, {gray_img.max()}]")
print(f"Mean brightness: {gray_img.mean():.1f}")

show_results(
    [rgb_img, gray_img, hsv_img[:,:,0]],
    ['Original RGB', 'Grayscale', 'HSV Hue Channel'],
    rows=1, cols=3
)

## 🔧 Step 2: Preprocessing - Enhancement and Noise Reduction

In [ ]:
# ========================================
# 2.1 Histogram Equalization for contrast enhancement
# ========================================
equalized = cv2.equalizeHist(gray_img)

# ========================================
# 2.2 Contrast Stretching
# ========================================
stretched = cv2.normalize(gray_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

# ========================================
# 2.3 Noise Reduction
# ========================================
# Gaussian blur for general noise
gaussian_blur = cv2.GaussianBlur(gray_img, (5, 5), 0)

# Median filter for salt-and-pepper noise
median_blur = cv2.medianBlur(gray_img, 5)

# Bilateral filter for edge-preserving smoothing
bilateral = cv2.bilateralFilter(gray_img, 9, 75, 75)

print("🔧 Preprocessing Results:")
print(f"Original contrast (std): {gray_img.std():.1f}")
print(f"Equalized contrast (std): {equalized.std():.1f}")
print(f"Stretched contrast (std): {stretched.std():.1f}")

show_results(
    [gray_img, equalized, stretched, gaussian_blur, median_blur, bilateral],
    ['Original', 'Equalized', 'Stretched', 'Gaussian Blur', 'Median Blur', 'Bilateral'],
    rows=2, cols=3
)

# Choose the best preprocessed image for further processing
# For object detection, bilateral filter often works best
processed_img = bilateral
print("✅ Selected bilateral filtered image for further processing")

## 🔒 Step 3: Thresholding - Separate Objects from Background

In [ ]:
# ========================================
# 3.1 Simple Thresholding
# ========================================
ret1, thresh_binary = cv2.threshold(processed_img, 127, 255, cv2.THRESH_BINARY)
ret2, thresh_binary_inv = cv2.threshold(processed_img, 127, 255, cv2.THRESH_BINARY_INV)

# ========================================
# 3.2 Otsu's Automatic Thresholding
# ========================================
ret_otsu, otsu_thresh = cv2.threshold(processed_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
ret_otsu_inv, otsu_thresh_inv = cv2.threshold(processed_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# ========================================
# 3.3 Adaptive Thresholding
# ========================================
adaptive_mean = cv2.adaptiveThreshold(processed_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
adaptive_gaussian = cv2.adaptiveThreshold(processed_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY, 11, 2)

print(f"🔒 Thresholding Analysis:")
print(f"Simple threshold value: {ret1}")
print(f"Otsu threshold value: {ret_otsu:.1f}")
print(f"Otsu inverted threshold: {ret_otsu_inv:.1f}")

show_results(
    [processed_img, thresh_binary, thresh_binary_inv, otsu_thresh, otsu_thresh_inv, adaptive_mean],
    ['Preprocessed', 'Binary (127)', 'Binary Inv (127)', 'Otsu Binary', 'Otsu Inv', 'Adaptive Mean'],
    rows=2, cols=3
)

# Choose the best thresholding result
# For object detection, usually Otsu or adaptive works best
# Let's analyze which gives better object separation
def count_objects(binary_img):
    """Count potential objects by finding contours"""
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Filter out very small contours (noise)
    significant_contours = [c for c in contours if cv2.contourArea(c) > 100]
    return len(significant_contours)

print(f"\n📊 Object Count Analysis:")
print(f"Binary (127): {count_objects(thresh_binary)} objects")
print(f"Otsu: {count_objects(otsu_thresh)} objects")
print(f"Otsu Inverted: {count_objects(otsu_thresh_inv)} objects")
print(f"Adaptive: {count_objects(adaptive_mean)} objects")

# Choose the best threshold (usually the one that detects reasonable number of objects)
thresholded_img = otsu_thresh_inv  # Often works well for bright objects on dark background
print("✅ Selected Otsu inverted thresholding for object detection")

## 🔧 Step 4: Morphological Operations - Clean Up Binary Masks

In [ ]:
# ========================================
# 4.1 Define Kernels
# ========================================
kernel_small = np.ones((3, 3), np.uint8)
kernel_medium = np.ones((5, 5), np.uint8)
kernel_large = np.ones((7, 7), np.uint8)
kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# ========================================
# 4.2 Basic Morphological Operations
# ========================================
# Remove small noise with opening (erosion followed by dilation)
opening = cv2.morphologyEx(thresholded_img, cv2.MORPH_OPEN, kernel_small)

# Fill small holes with closing (dilation followed by erosion)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_medium)

# Further noise removal
final_opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel_ellipse)

# ========================================
# 4.3 Advanced Morphological Operations
# ========================================
# Morphological gradient for edge detection
gradient = cv2.morphologyEx(thresholded_img, cv2.MORPH_GRADIENT, kernel_small)

# Top hat for small bright details
tophat = cv2.morphologyEx(thresholded_img, cv2.MORPH_TOPHAT, kernel_medium)

print("🔧 Morphological Cleanup Results:")
print(f"Original objects: {count_objects(thresholded_img)}")
print(f"After opening: {count_objects(opening)}")
print(f"After closing: {count_objects(closing)}")
print(f"Final cleaned: {count_objects(final_opening)}")

show_results(
    [thresholded_img, opening, closing, final_opening, gradient, tophat],
    ['Original Binary', 'Opening', 'Closing', 'Final Cleaned', 'Gradient', 'Top Hat'],
    rows=2, cols=3
)

# Use the cleaned binary image for contour detection
cleaned_binary = final_opening
print("✅ Binary mask cleaned and ready for contour detection")

## 🎯 Step 5: Contour Detection and Object Analysis

In [ ]:
# ========================================
# 5.1 Find Contours
# ========================================
contours, hierarchy = cv2.findContours(cleaned_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours by area to remove noise
min_area = 100
significant_contours = [c for c in contours if cv2.contourArea(c) > min_area]

print(f"🎯 Contour Detection Results:")
print(f"Total contours found: {len(contours)}")
print(f"Significant contours (area > {min_area}): {len(significant_contours)}")

# ========================================
# 5.2 Draw Contours and Bounding Boxes
# ========================================
# Create visualization images
contour_img = cv2.cvtColor(cleaned_binary, cv2.COLOR_GRAY2BGR)
bbox_img = rgb_img.copy()
analysis_img = rgb_img.copy()

# Draw contours in different colors
cv2.drawContours(contour_img, significant_contours, -1, (0, 255, 0), 2)

# ========================================
# 5.3 Object Analysis
# ========================================
object_data = []

for i, contour in enumerate(significant_contours):
    # Calculate properties
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    # Bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(bbox_img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.putText(bbox_img, f'Obj {i+1}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
    # Minimum enclosing circle
    (cx, cy), radius = cv2.minEnclosingCircle(contour)
    center = (int(cx), int(cy))
    radius = int(radius)
    cv2.circle(analysis_img, center, radius, (0, 255, 255), 2)
    cv2.circle(analysis_img, center, 2, (0, 0, 255), -1)
    
    # Fit ellipse (if contour has enough points)
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        cv2.ellipse(analysis_img, ellipse, (255, 0, 255), 1)
    
    # Calculate additional properties
    aspect_ratio = float(w) / h
    extent = float(area) / (w * h)
    solidity = float(area) / cv2.contourArea(cv2.convexHull(contour))
    
    object_data.append({
        'id': i + 1,
        'area': area,
        'perimeter': perimeter,
        'aspect_ratio': aspect_ratio,
        'extent': extent,
        'solidity': solidity,
        'center': center
    })
    
    print(f"Object {i+1}:")
    print(f"  Area: {area:.0f} pixels")
    print(f"  Perimeter: {perimeter:.1f} pixels")
    print(f"  Aspect Ratio: {aspect_ratio:.2f}")
    print(f"  Extent: {extent:.2f}")
    print(f"  Solidity: {solidity:.2f}")
    print(f"  Center: {center}")
    print()

show_results(
    [rgb_img, cv2.cvtColor(contour_img, cv2.COLOR_BGR2RGB), bbox_img, analysis_img],
    ['Original Image', 'Detected Contours', 'Bounding Boxes', 'Shape Analysis'],
    rows=2, cols=2
)

## 📊 Step 6: Object Classification and Final Results

In [ ]:
# ========================================
# 6.1 Simple Object Classification
# ========================================
def classify_object(obj_data):
    """Simple classification based on shape properties"""
    aspect_ratio = obj_data['aspect_ratio']
    extent = obj_data['extent']
    solidity = obj_data['solidity']
    
    if extent > 0.9 and solidity > 0.9:
        if 0.8 < aspect_ratio < 1.2:
            return "Square/Rectangle"
        else:
            return "Rectangle"
    elif solidity > 0.8 and extent < 0.8:
        return "Circle/Ellipse"
    elif solidity < 0.8:
        return "Complex Shape"
    else:
        return "Unknown"

# Classify all detected objects
classified_img = rgb_img.copy()

print("📊 Object Classification Results:")
print("=" * 40)

for i, obj in enumerate(object_data):
    classification = classify_object(obj)
    obj['classification'] = classification
    
    # Draw classification on image
    center = obj['center']
    cv2.putText(classified_img, f"{obj['id']}: {classification}", 
                (center[0]-50, center[1]+20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
    
    print(f"Object {obj['id']}: {classification}")
    print(f"  Properties: AR={obj['aspect_ratio']:.2f}, Ext={obj['extent']:.2f}, Sol={obj['solidity']:.2f}")

# ========================================
# 6.2 Create Object Masks
# ========================================
individual_masks = []
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]

colored_objects = np.zeros_like(rgb_img)

for i, contour in enumerate(significant_contours):
    # Create individual mask for each object
    mask = np.zeros(cleaned_binary.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    individual_masks.append(mask)
    
    # Color each object differently
    color = colors[i % len(colors)]
    colored_objects[mask > 0] = color

# ========================================
# 6.3 Final Results Summary
# ========================================
print(f"\n🎯 Detection Pipeline Summary:")
print("=" * 40)
print(f"Total objects detected: {len(significant_contours)}")
print(f"Average object area: {np.mean([obj['area'] for obj in object_data]):.0f} pixels")
print(f"Largest object area: {max([obj['area'] for obj in object_data]):.0f} pixels")
print(f"Smallest object area: {min([obj['area'] for obj in object_data]):.0f} pixels")

# Count classifications
classifications = [obj['classification'] for obj in object_data]
unique_classes = set(classifications)
for cls in unique_classes:
    count = classifications.count(cls)
    print(f"{cls}: {count} objects")

show_results(
    [rgb_img, cleaned_binary, classified_img, colored_objects],
    ['Original Image', 'Final Binary Mask', 'Classified Objects', 'Colored Objects'],
    rows=2, cols=2
)

print("\n✅ Object Detection and Segmentation Pipeline Completed Successfully!")
print("📚 This pipeline demonstrates:")
print("   - Image preprocessing and enhancement")
print("   - Thresholding techniques for segmentation")
print("   - Morphological operations for cleanup")
print("   - Contour detection and analysis")
print("   - Object property extraction and classification")